In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score

In [2]:
df_accidents = pd.read_csv('../data/MA3_finished.csv', index_col='ID')

In [3]:
#Daten in Trainigs/Test/Entwicklungs Daten aufteilen

#featureselection
obj_bool_features = df_accidents[['Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit',
                         'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
                         'Sunrise_Sunset']]
#encode features above
le = LabelEncoder()
#create copy of df
df_encoded = pd.DataFrame()
#LabelEncoder can only encode one column at a time --> forloop
#obj_bool_feat = le.fit_transform(obj_bool_features)
for feature in obj_bool_features:
    df_encoded[feature] = le.fit_transform(df_accidents[feature])

#float/int features
float_int_features = df_accidents[['Severity','Start_Lng', 'Start_Lat', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)',]]

for feature in float_int_features:
    df_encoded[feature] = df_accidents[feature]

#set Target
target = df_encoded['Severity']
#set features
features = df_encoded.drop(columns=['Severity'])
#scale features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

#Split Trainings/Test Daten 80/20 
features_train, features_test1, target_train, target_test1 = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

#Split Test in Entwicklungs und echten Daten 50/50
features_dev, features_test, target_dev, target_test = train_test_split(features_test1, target_test1, test_size=0.5, random_state=42)

In [4]:
# Übersicht über Klassen in den Trainingsdaten

print("Gesamtzahl Datenpunkte Training: ", len(target_train))

# Range(0,4) gibt bei Kategorie 0 = 0
# Range (1,6) gibt bei Katg. 5 = 0
# deswegen (1,5) (4 Severitys)
for i in range(1,5):
    print("Anzahl Kategorie", str(i), list(target_train).count(i))

Gesamtzahl Datenpunkte Training:  5641244
Anzahl Kategorie 1 51977
Anzahl Kategorie 2 4537583
Anzahl Kategorie 3 908712
Anzahl Kategorie 4 142972


In [6]:
# Entwicklungsdaten
print("Gesamtzahl Datenpunkte Dev: ", len(target_dev))
for i in range(1,5):
    print("Anzahl Kategorie", str(i), list(target_dev).count(i))

Gesamtzahl Datenpunkte Dev:  705156
Anzahl Kategorie 1 6635
Anzahl Kategorie 2 567067
Anzahl Kategorie 3 113583
Anzahl Kategorie 4 17871


In [7]:
# Vorhersage-Array bauen: Für jedes Element der Entwicklungsdaten Klasse '2' (Bei uns Severity-Stufe 2) vorhersagen
pred = np.full(len(target_dev),2)

print(len(target_dev), len(pred))

#Berechnung f1-score
f1 = f1_score(target_test, pred, average='weighted')

print("Der gewichtete F1-Score für die konstante Vorhersage '2' ist: ", f1)

705156 705156
Der gewichtete F1-Score für die konstante Vorhersage '2' ist:  0.7164639019274487


In [ ]:
#f1-score von 0.71 ist annehmbar --> gute Blaance zwischen Präzision und Recall (Der Wert könnte weiterhoch gepusht werden indem
# mehr Outliner eliminert werden wprden ODER Modellwechsel ODER featureauswahl verbessern)